<a href="https://colab.research.google.com/github/Get-the-CODE/SQL-Codes/blob/main/Sumit_Agarwal_PG24_3A_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
Tutorial based on [SQLCourse2](https://www.sqlcourse2.com/) <br>


In [ ]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-02-20 19:26:52.988199+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

In [ ]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!pip install psycopg2
import psycopg2
import pandas as pd
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


#Create Schema, Tables and Insert Data

In [ ]:
# This installation creates an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:postgres:pass
localhost:5432:dbpraxis:upraxis:upass


In [ ]:
# Download two CSV files from G-Drive into the Colab VM

#!gdown -qq 1Ug15GbEmeTnwJnuH-4fpPeZ1k20ryoGx      # Quiet Mode, not output
#!gdown -qq 1qx6duSqzQUJSkpHNQIIZ9l0T6zYtjT-y      # Quiet Mode, not output

!wget -O Customers.csv -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SQLCourse2%20-%20Customers.csv
!wget -O ItemsOrdered.csv -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SQLCourse2%20-%20ItemsOrdered.csv

In [ ]:
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists itemsordered'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS itemsordered (\
customerid char(5),orderdate date, item varchar(30), quantity int, price decimal(6,2)) \
'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists customers'
# --------------------------------------------------------------
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\
CREATE TABLE IF NOT EXISTS customers (customerid char(5),firstname varchar(20),lastname varchar(20), city varchar(20), state varchar(20)) \
'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dt+'

NOTICE:  table "itemsordered" does not exist, skipping
DROP TABLE
CREATE TABLE
NOTICE:  table "customers" does not exist, skipping
DROP TABLE
CREATE TABLE
                                       List of relations
 Schema |     Name     | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+--------------+-------+---------+-------------+---------------+---------+-------------
 public | customers    | table | upraxis | permanent   | heap          | 0 bytes | 
 public | itemsordered | table | upraxis | permanent   | heap          | 0 bytes | 
(2 rows)



In [ ]:
# Only superuser ('postgres') can use copy command, not even database owner ('upraxis')
# 'postgres' password has to be entered manually, password = 'pass'
#
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY customers FROM '/content/Customers.csv' DELIMITER ',' CSV HEADER;"
!psql -h localhost -p 5432 -Upostgres -ddbpraxis -c "COPY itemsordered FROM '/content/ItemsOrdered.csv' DELIMITER ',' CSV HEADER;"

COPY 17
COPY 32


In [ ]:
#!cat ItemsOrdered.csv
#!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'select * from itemsordered'

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [ ]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [ ]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [ ]:
#runSelect("select * from itemsordered")
runSelect("select * from customers")

customerid,firstname,lastname,city,state
10101,John,Gray,Lynden,Washington
10298,Leroy,Brown,Pinetop,Arizona
10299,Elroy,Keller,Snoqualmie,Washington
10315,Lisa,Jones,Oshkosh,Wisconsin
10325,Ginger,Schultz,Pocatello,Idaho
10329,Kelly,Mendoza,Kailua,Hawaii
10330,Shawn,Dalton,Cannon Beach,Oregon
10338,Michael,Howell,Tillamook,Oregon
10339,Anthony,Sanchez,Winslow,Arizona
10408,Elroy,Cleaver,Globe,Arizona


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-02-20 19:27:16.022661+05:30


# EXCERCISE STARTS FROM HERE:

## 1 SELECT

In [ ]:
#runSelect("select * from itemsordered")
runSelect("select * from customers limit 5")

customerid,firstname,lastname,city,state
10101,John,Gray,Lynden,Washington
10298,Leroy,Brown,Pinetop,Arizona
10299,Elroy,Keller,Snoqualmie,Washington
10315,Lisa,Jones,Oshkosh,Wisconsin
10325,Ginger,Schultz,Pocatello,Idaho


In [ ]:
runSelect("select * from itemsordered limit 5")
#runSelect("select * from customers")

customerid,orderdate,item,quantity,price
10330,1999-06-30,Pogo stick,1,28.00
10101,1999-06-30,Raft,1,58.00
10298,1999-07-01,Skateboard,1,33.00
10101,1999-07-01,Life Vest,4,125.00
10299,1999-07-06,Parachute,1,1250.00


In [ ]:
#From the items_ordered table, select a list of all items purchased for customerid 10449.
#Display the customerid, item, and price for this customer.

runSelect("select customerid, item, price from itemsordered where customerid='10449'")

customerid,item,price
10449,Unicycle,180.79
10449,Snow Shoes,45.00
10449,Bicycle,380.50
10449,Canoe,280.00
10449,Flashlight,4.50
10449,Canoe paddle,40.00


In [ ]:
# Select all columns from the items_ordered table for whoever purchased a Tent.

runSelect("select * from itemsordered where item='Tent'")

customerid,orderdate,item,quantity,price
10439,1999-09-18,Tent,1,88.00
10438,2000-01-18,Tent,1,79.99


In [ ]:
#Select the customerid, order_date, and item values from the items_ordered table
#for any items in the item column that start with the letter “S”.

runSelect("select customerid, orderdate, price, item from itemsordered where item like 'S%'")

customerid,orderdate,price,item
10298,1999-07-01,33.00,Skateboard
10439,1999-08-14,25.50,Ski Poles
10449,1999-09-01,45.00,Snow Shoes
10410,1999-10-28,89.22,Sleeping Bag
10101,2000-03-08,88.70,Sleeping Bag
10330,2000-04-19,16.75,Shovel


In [ ]:
#Select the distinct items in the items_ordered table. In other words, display a listing of each of the unique items from the items_ordered table.

runSelect("select distinct(item) from itemsordered")

item
Parachute
Canoe
Flashlight
Shovel
Umbrella
Ear Muffs
Inflatable Mattress
Ski Poles
Life Vest
Skateboard


In [ ]:
#Make up your own select statements and submit them.

runSelect("select * from itemsordered where orderdate not between '1999-01-01' and '1999-12-31'")

customerid,orderdate,item,quantity,price
10330,2000-01-01,Flashlight,4,28.00
10101,2000-01-02,Lantern,1,16.00
10299,2000-01-18,Inflatable Mattress,1,38.00
10438,2000-01-18,Tent,1,79.99
10413,2000-01-19,Lawnchair,4,32.00
10410,2000-01-30,Unicycle,1,192.50
10315,2000-02-02,Compass,1,8.00
10449,2000-02-29,Flashlight,1,4.50
10101,2000-03-08,Sleeping Bag,2,88.70
10298,2000-03-18,Pocket Knife,1,22.38


##2 AGGREGATE FUNCTIONS

In [ ]:
#Select the maximum price of any item ordered in the items_ordered table. Hint: Select the maximum price only.

runSelect("select max(price) from itemsordered")

max
1250.00


In [ ]:
#Select the average price of all of the items ordered that were purchased in the month of Dec.

runSelect("select item, ROUND(avg(price),2) from itemsordered where to_char(orderdate,'mm')='12'group by item")


item,round
Bicycle,380.50
Canoe,280.00
Helmet,22.00
Hoola Hoop,14.75


In [ ]:
#What are the total number of rows in the items_ordered table?

runSelect("select count(*) from itemsordered")

count
32


In [ ]:
#For all of the tents that were ordered in the items_ordered table,
#what is the price of the lowest tent? Hint: Your query should return the price only.

runSelect("select min(price) from itemsordered where item in ('Tent')")


min
79.99


##3 GROUP BY

In [ ]:
#How many people are in each unique state in the customers table? Select the state and
#display the number of people in each. Hint: count is used to count rows in a column,
#sum works on numeric data only.
#runSelect("select * from customers limit 5")

runSelect("select state, count(distinct(customerid)) from customers group by state")

state,count
Arizona,6
Colorado,2
Hawaii,1
Idaho,1
North Carolina,1
Oregon,2
South Carolina,1
Washington,2
Wisconsin,1


In [ ]:
#From the items_ordered table, select the item, maximum price, and minimum price for each specific item in the
#table. Hint: The items will need to be broken up into separate groups.


runSelect("select item, max(price) Max_price_of_item, min(price) Min_price_of_item from itemsordered group by item")

item,max_price_of_item,min_price_of_item
Parachute,1250.00,1250.00
Canoe,280.00,280.00
Flashlight,28.00,4.50
Shovel,16.75,16.75
Umbrella,6.75,4.50
Ear Muffs,12.50,12.50
Inflatable Mattress,38.00,38.00
Ski Poles,25.50,25.50
Life Vest,125.00,125.00
Skateboard,33.00,33.00


In [ ]:
#How many orders did each customer make? Use the items_ordered table. Select the customerid,
#number of orders they made, and the sum of their orders. Click the Group By answers link below if you have any problems.

runSelect("select customerid, count(customerid) No_of_orders, sum(price) Total_order_value from itemsordered group by customerid")

customerid,no_of_orders,total_order_value
10299,2,1288.00
10410,2,281.72
10339,1,4.50
10298,5,118.88
10330,3,72.75
10438,3,95.24
10413,1,32.00
10439,2,113.50
10101,6,320.75
10449,6,930.79


##4 HAVING

In [ ]:
#How many people are in each unique state in the customers table that have more than one person in the state?
#Select the state and display the number of how many people are in each if it’s greater than 1.

runSelect("select state, count(distinct(customerid)) from customers group by state having count(distinct(customerid))>1")

state,count
Arizona,6
Colorado,2
Oregon,2
Washington,2


In [ ]:
#From the items_ordered table, select the item, maximum price, and minimum price for each
#specific item in the table. Only display the results if the maximum price for one of the items is greater than 190.00.

runSelect("select item, max(price), min(price) from itemsordered group by item having max(price)>190")

item,max,min
Parachute,1250.00,1250.00
Canoe,280.00,280.00
Bicycle,380.50,380.50
Unicycle,192.50,180.79


In [ ]:
#How many orders did each customer make? Use the items_ordered table. Select the customerid,
#number of orders they made, and the sum of their orders if they purchased more than 1 item.

runSelect("select customerid, count(customerid) No_of_orders, sum(price) Total_order_value from itemsordered group by customerid having count(customerid)>1")

customerid,no_of_orders,total_order_value
10299,2,1288.00
10410,2,281.72
10298,5,118.88
10330,3,72.75
10438,3,95.24
10439,2,113.50
10101,6,320.75
10449,6,930.79


##5 ORDER BY

In [ ]:
#Select the lastname, firstname, and city for all customers in the customers table.
#Display the results in Ascending Order based on the lastname.

runSelect("select firstname, lastname, city from customers order by lastname")

firstname,lastname,city
Leroy,Brown,Pinetop
Elroy,Cleaver,Globe
Shawn,Dalton,Cannon Beach
Donald,Davids,Gila Bend
Conrad,Giles,Telluride
Sarah,Graham,Greensboro
John,Gray,Lynden
Michael,Howell,Tillamook
Mary Ann,Howell,Charleston
Lisa,Jones,Oshkosh


In [ ]:
#Same thing as exercise #1, but display the results in Descending order.

runSelect("select firstname, lastname, city from customers order by lastname desc")

firstname,lastname,city
Kevin,Smith,Durango
Ginger,Schultz,Pocatello
Anthony,Sanchez,Winslow
Linda,Sakahara,Nogales
Isabela,Moore,Yuma
Kelly,Mendoza,Kailua
Elroy,Keller,Snoqualmie
Lisa,Jones,Oshkosh
Mary Ann,Howell,Charleston
Michael,Howell,Tillamook


In [ ]:
#Select the item and price for all of the items in the items_ordered table that the
#price is greater than 10.00. Display the results in Ascending order based on the price.

runSelect("select item, price from itemsordered where price>10 order by price")

item,price
Ear Muffs,12.50
Hoola Hoop,14.75
Lantern,16.00
Shovel,16.75
Rain Coat,18.30
Helmet,22.00
Pocket Knife,22.38
Ski Poles,25.50
Pogo stick,28.00
Flashlight,28.00


##6 COMBINING CONDITIONS & BOOLEAN OPERATORS

In [ ]:
#Select the customerid, order_date, and item from the items_ordered table for all items unless they are
#‘Snow Shoes’ or if they are ‘Ear Muffs’. Display the rows as long as they are not either of these two items.

runSelect("select customerid, orderdate, item from itemsordered where item not in ('Snow Shoes', 'Ear Muffs')")

customerid,orderdate,item
10330,1999-06-30,Pogo stick
10101,1999-06-30,Raft
10298,1999-07-01,Skateboard
10101,1999-07-01,Life Vest
10299,1999-07-06,Parachute
10339,1999-07-27,Umbrella
10449,1999-08-13,Unicycle
10439,1999-08-14,Ski Poles
10101,1999-08-18,Rain Coat
10439,1999-09-18,Tent


In [ ]:
#Select the item and price of all items that start with the letters ‘S’, ‘P’, or ‘F’.

runSelect("select item, price from itemsordered where item like 'S%' or item like 'P%' or item like 'F%'")

item,price
Pogo stick,28.00
Skateboard,33.00
Parachute,1250.00
Ski Poles,25.50
Snow Shoes,45.00
Sleeping Bag,89.22
Pillow,8.50
Flashlight,28.00
Flashlight,4.50
Sleeping Bag,88.70


##7 IN & BETWEEN

In [ ]:
#Select the date, item, and price from the items_ordered table for all of the rows
#that have a price value ranging from 10.00 to 80.00.

runSelect("select orderdate, item, price from itemsordered where price between 10 and 80")

orderdate,item,price
1999-06-30,Pogo stick,28.00
1999-06-30,Raft,58.00
1999-07-01,Skateboard,33.00
1999-08-14,Ski Poles,25.50
1999-08-18,Rain Coat,18.30
1999-09-01,Snow Shoes,45.00
1999-09-19,Lantern,29.00
1999-12-01,Helmet,22.00
1999-12-30,Hoola Hoop,14.75
2000-01-01,Flashlight,28.00


In [ ]:
#Select the firstname, city, and state from the customers table for all of the rows where the state value is either:
#Arizona, Washington, Oklahoma, Colorado, or Hawaii.

runSelect("select firstname, city, state from customers where state in ('Arizona', 'Washington', 'Oklahoma', 'Colorado', 'Hawaii')")

firstname,city,state
John,Lynden,Washington
Leroy,Pinetop,Arizona
Elroy,Snoqualmie,Washington
Kelly,Kailua,Hawaii
Anthony,Winslow,Arizona
Elroy,Globe,Arizona
Donald,Gila Bend,Arizona
Linda,Nogales,Arizona
Kevin,Durango,Colorado
Conrad,Telluride,Colorado


##8 MATHEMATICAL FUNCTIONS

In [ ]:
#Select the item and per unit price for each item in the items_ordered table. Hint: Divide the price by the quantity.

runSelect("select distinct(item), round(price/quantity,2) as per_unit_price from itemsordered")

item,per_unit_price
Helmet,22.00
Rain Coat,18.30
Flashlight,4.50
Inflatable Mattress,38.00
Lantern,16.00
Tent,79.99
Unicycle,192.50
Life Vest,31.25
Pocket Knife,22.38
Hoola Hoop,4.92


##9 TABLE JOINS, A MUST

In [ ]:
#Write a query using a join to determine which items were ordered by each of the customers
#in the customers table. Select the customerid, firstname, lastname, order_date, item, and
# price for everything each customer purchased in the items_ordered table.

In [ ]:
runSelect("select i.customerid, firstname, lastname, orderdate, item, price from customers c right join itemsordered i on c.customerid=i.customerid order by i.customerid, orderdate, price desc")

customerid,firstname,lastname,orderdate,item,price
10101,John,Gray,1999-06-30,Raft,58.00
10101,John,Gray,1999-07-01,Life Vest,125.00
10101,John,Gray,1999-08-18,Rain Coat,18.30
10101,John,Gray,1999-12-30,Hoola Hoop,14.75
10101,John,Gray,2000-01-02,Lantern,16.00
10101,John,Gray,2000-03-08,Sleeping Bag,88.70
10298,Leroy,Brown,1999-07-01,Skateboard,33.00
10298,Leroy,Brown,1999-09-19,Lantern,29.00
10298,Leroy,Brown,1999-12-01,Helmet,22.00
10298,Leroy,Brown,2000-03-18,Pocket Knife,22.38


In [ ]:
#Repeat exercise #1, however display the results sorted by state in descending order.

runSelect("select i.customerid, firstname, lastname, state, orderdate, item, price from customers c right join itemsordered i on c.customerid=i.customerid order by state desc")

customerid,firstname,lastname,state,orderdate,item,price
10315,Lisa,Jones,Wisconsin,2000-02-02,Compass,8.00
10101,John,Gray,Washington,1999-06-30,Raft,58.00
10101,John,Gray,Washington,2000-03-08,Sleeping Bag,88.70
10101,John,Gray,Washington,1999-07-01,Life Vest,125.00
10299,Elroy,Keller,Washington,1999-07-06,Parachute,1250.00
10101,John,Gray,Washington,1999-12-30,Hoola Hoop,14.75
10101,John,Gray,Washington,1999-08-18,Rain Coat,18.30
10101,John,Gray,Washington,2000-01-02,Lantern,16.00
10299,Elroy,Keller,Washington,2000-01-18,Inflatable Mattress,38.00
10410,Mary Ann,Howell,South Carolina,2000-01-30,Unicycle,192.50


# END